In [1]:
import sys
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '5' 
from os.path import abspath
import numpy as np
import pandas as pd
from utils.generate_network import generate_network
from utils.prepare_data import prepare_data
from utils.popphy_io import get_config, save_params, load_params
from utils.popphy_io import get_stat, get_stat_dict
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from models.PopPhy import PopPhyCNN
from models.CNN1D import CNN1D
from models.MLPNN import MLPNN
from models.RF import RF
from models.SVM import SVM
from models.LASSO import LASSO
import warnings
from datetime import datetime
import json
import warnings
warnings.filterwarnings("ignore")

/opt/anaconda3/envs/python3.8/lib/python3.8/site-packages/scipy/__init__.py:143: UserWarning: A NumPy version >=1.19.5 and <1.27.0 is required for this version of SciPy (detected version 1.19.0)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
config = get_config()
filt_thresh = config.get('Evaluation', 'FilterThresh')
dataset = config.get('Evaluation', 'DataSet')
num_runs = int(config.get('Evaluation', 'NumberRuns'))
num_test = int(config.get('Evaluation', 'NumberTestSplits'))
path = "../data/" + dataset 

print(dataset)

T2D


In [3]:
results_dir = "../results/notebook_results/" + dataset

try:
    os.makedirs(results_dir)
except OSError:
    print ("Creation of the directory %s failed" % results_dir)
else:
    print ("Successfully created the directory %s" % results_dir)

Creation of the directory ../results/notebook_results/T2D failed


In [4]:
my_maps, raw_x, tree_x, raw_features, tree_features, labels, label_set, g, feature_df = prepare_data(path, config)

num_class = len(np.unique(labels))
if num_class == 2:
    metric = "AUC"
else:
    metric = "MCC"

seed = np.random.randint(100)
np.random.seed(seed)
np.random.shuffle(my_maps)
np.random.seed(seed)
np.random.shuffle(raw_x)
np.random.seed(seed)
np.random.shuffle(tree_x)
np.random.seed(seed)
np.random.shuffle(labels)

n_values = np.max(labels) + 1
labels_oh = np.eye(n_values)[labels]

tree_row = my_maps.shape[1]
tree_col = my_maps.shape[2]

print("There are %d classes...%s" % (num_class, ", ".join(label_set)))
cv_list = ["Run_" + str(x) + "_CV_" + str(y) for x in range(num_runs) for y in range(num_test)]
seeds = np.random.randint(1000, size=num_runs)

There are 216 raw features...
Building tree structure...
Found tree file...
Populating trees...
There are 406 tree features...
There are 2 classes...n, t2d


In [5]:
rf_tree_stat_df = pd.DataFrame(index=["AUC", "MCC", "Precision", "Recall", "F1"], columns=cv_list)

run = 0
for seed in seeds:
    skf = StratifiedKFold(n_splits=num_test, shuffle=True, random_state=seed)
    fold = 0
    for train_index, test_index in skf.split(my_maps, labels):
        train_x, test_x = tree_x[train_index,:], tree_x[test_index,:]
        train_y_oh, test_y_oh = labels_oh[train_index,:], labels_oh[test_index,:]
        train_y, test_y = labels[train_index], labels[test_index]
        
        train_x = np.log(train_x + 1)
        test_x = np.log(test_x + 1)
        
        c_prob = [0] * len(np.unique(labels))
        train_weights = []

        for l in np.unique(labels):
            a = float(len(labels))
            b = 2.0 * float((np.sum(labels==l)))
            c_prob[int(l)] = a/b

        c_prob = np.array(c_prob).reshape(-1)

        for l in np.argmax(train_y_oh, 1):
            train_weights.append(c_prob[int(l)])
        train_weights = np.array(train_weights)
        
        scaler = MinMaxScaler().fit(train_x)
        train_x = np.clip(scaler.transform(train_x), 0, 1)
        test_x = np.clip(scaler.transform(test_x), 0, 1) 

        train_oh = [train_x, train_y_oh]
        test_oh = [test_x, test_y_oh]

        train = [train_x, train_y]
        test = [test_x, test_y]
        
        rf_model = RF(config)
        
        rf_model.train(train)
        preds, rf_stats = rf_model.test(test)
        if num_class == 2:
                rf_tree_stat_df.loc["AUC"]["Run_" + str(run) + "_CV_" + str(fold)]=rf_stats["AUC"]
        rf_tree_stat_df.loc["MCC"]["Run_" + str(run) + "_CV_" + str(fold)]=rf_stats["MCC"]
        rf_tree_stat_df.loc["Precision"]["Run_" + str(run) + "_CV_" + str(fold)]=rf_stats["Precision"]
        rf_tree_stat_df.loc["Recall"]["Run_" + str(run) + "_CV_" + str(fold)]=rf_stats["Recall"]
        rf_tree_stat_df.loc["F1"]["Run_" + str(run) + "_CV_" + str(fold)]=rf_stats["F1"]
               

        del(rf_model)
        
        fold += 1
    run += 1

In [6]:
rf_tree_stat_df.to_csv(os.path.join(results_dir, "rf_tree_evaluation.csv"))
rf_tree_stat_df 

,Run_0_CV_0,Run_0_CV_1,Run_0_CV_2,Run_0_CV_3,Run_0_CV_4,Run_0_CV_5,Run_0_CV_6,Run_0_CV_7,Run_0_CV_8,Run_0_CV_9
AUC,0.805383,0.716356,0.729814,0.809917,0.714876,0.806818,0.771694,0.678719,0.627066,0.667355
MCC,0.456624,0.271222,0.271222,0.500517,0.229416,0.504715,0.29277,0.229416,0.182574,0.181818
Precision,0.732818,0.636364,0.636364,0.750518,0.615789,0.754737,0.657143,0.615789,0.591667,0.590909
Recall,0.727273,0.636364,0.636364,0.75,0.613636,0.75,0.636364,0.613636,0.590909,0.590909
F1,0.723828,0.636364,0.636364,0.749871,0.611832,0.748832,0.623932,0.611832,0.590062,0.590909


In [7]:
rf_tree_stat_df_mean = rf_tree_stat_df.mean(1)

rf_tree_stat_df_mean

AUC          0.732800
MCC          0.312029
Precision    0.658210
Recall       0.654545
F1           0.652382
dtype: float64